In [ ]:
# Untuk membaca file Excel
import pandas as pd

# FUNGSI KEANGGOTAAN
def triangular(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x < c:
        return (c - x) / (c - b)
    else:
        return 0.0

def trapezoidal(x, a, b, c, d):
    if x <= a:
        return 1.0 if a == b else 0.0
    elif a < x < b:
        return (x - a) / (b - a) if a != b else 1.0
    elif b <= x <= c:
        return 1.0
    elif c < x < d:
        return (d - x) / (d - c)
    else:
        return 1.0 if c == d else 0.0



# FUZZIFIKASI INPUT
def fuzzify_gaji(gaji):
    return {
        "rendah": trapezoidal(gaji, 0, 0, 0, 6),
        "sedang": triangular(gaji, 4, 8, 12),
        "tinggi": trapezoidal(gaji, 10, 15, 30, 30)
    }

def fuzzify_cicilan(cicilan):
    return {
        "kecil": trapezoidal(cicilan, 0, 0, 0, 15),
        "sedang": triangular(cicilan, 25, 40, 55),
        "besar": trapezoidal(cicilan, 45, 55, 70, 70)
    }


# ATURAN FUZZY
aturan = [
    ("tinggi", "kecil", "rendah"),
    ("tinggi", "sedang", "sedang"),
    ("tinggi", "besar", "sedang"),
    ("sedang", "kecil", "rendah"),
    ("sedang", "sedang", "sedang"),
    ("sedang", "besar", "tinggi"),
    ("rendah", "kecil", "sedang"),
    ("rendah", "sedang", "tinggi"),
    ("rendah", "besar", "tinggi"),
]



# FUNGSI KEANGGOTAAN OUTPUT
def risiko_rendah(x):
    return trapezoidal(x, 0, 0, 0, 40)

def risiko_sedang(x):
    return triangular(x, 30, 50, 70)

def risiko_tinggi(x):
    return trapezoidal(x, 60, 80, 100, 100)


# INFERENSI MAMDANI
def inferensi_mamdani(gaji_fz, cicilan_fz):
    domain = range(0, 101)
    agregasi = [0.0] * 101

    for g, c, r in aturan:
        alpha = min(gaji_fz[g], cicilan_fz[c])
        for x in domain:
            if r == "rendah":
                mu = risiko_rendah(x)
            elif r == "sedang":
                mu = risiko_sedang(x)
            else:
                mu = risiko_tinggi(x)
            agregasi[x] = max(agregasi[x], min(alpha, mu))

    pembilang = sum(x * agregasi[x] for x in domain)
    penyebut = sum(agregasi)
    return pembilang / penyebut if penyebut != 0 else 0.0


# OUTPUT MONOTON UNTUK TSUKAMOTO
def risiko_rendah_tsukamoto(alpha):
    return 50 - 20 * alpha

def risiko_tinggi_tsukamoto(alpha):
    return 50 + 20 * alpha


# INFERENSI TSUKAMOTO
def inferensi_tsukamoto(gaji_fz, cicilan_fz):
    pembilang = 0.0
    penyebut = 0.0

    for g, c, r in aturan:
        alpha = min(gaji_fz[g], cicilan_fz[c])
        if alpha == 0:
            continue
        if r == "rendah":
            z = risiko_rendah_tsukamoto(alpha)
        elif r == "tinggi":
            z = risiko_tinggi_tsukamoto(alpha)
        else:
            z = 50
        pembilang += alpha * z
        penyebut += alpha

    return pembilang / penyebut if penyebut != 0 else 0.0


# TOP 10 TERENDAH DAN SIMPAN FILE
def top_10_terendah(hasil, nama_metode="Mamdani"):
    hasil_sorted = sorted(hasil, key=lambda x: x[1])
    top10 = hasil_sorted[:10]

    print(f"\n10 Nilai Kelayakan Terendah - {nama_metode}")
    print("-" * 40)
    print("ID | Nilai")
    print("-" * 40)
    for id_p, nilai in top10:
        print(f"{id_p:3} | {nilai:6.2f}")

    # Simpan ke file Excel
    df_top10 = pd.DataFrame(top10, columns=["ID", "Nilai"])
    df_top10.to_excel(f"top10_{nama_metode.lower()}.xlsx", index=False)


# MAIN PROGRAM
def main():
    # Baca data Excel, header mulai dari baris kedua (0-indexed -> header=1)
    df = pd.read_excel("resiko kredit.xlsx", header=1)

    hasil_mamdani = []
    hasil_tsukamoto = []

    print("ID | Mamdani | Tsukamoto")
    print("-" * 30)

    for _, row in df.iterrows():
        id_p = int(row[0])
        gaji = float(row[1])
        cicilan = float(row[2])

        g_fz = fuzzify_gaji(gaji)
        c_fz = fuzzify_cicilan(cicilan)

        m = inferensi_mamdani(g_fz, c_fz)
        t = inferensi_tsukamoto(g_fz, c_fz)

        hasil_mamdani.append((id_p, m))
        hasil_tsukamoto.append((id_p, t))

        print(f"{id_p:2d} | {m:7.2f} | {t:9.2f}")

    # Tampilkan dan simpan 10 nilai terendah
    top_10_terendah(hasil_mamdani, "Mamdani")
    top_10_terendah(hasil_tsukamoto, "Tsukamoto")

if __name__ == "__main__":
    main()

ID | Mamdani | Tsukamoto
------------------------------
 1 |   82.11 |     56.73
 2 |   63.91 |     52.35
 3 |   82.71 |     60.33
 4 |   50.00 |     50.00
 5 |   79.63 |     54.97
 6 |   50.00 |     50.00
 7 |   58.79 |     51.05
 8 |   50.00 |     50.00
 9 |   82.56 |     59.67
10 |   50.00 |     50.00
11 |   83.44 |     63.67
12 |   50.00 |     50.00
13 |   50.00 |     50.00
14 |   50.00 |     50.00
15 |   50.00 |     50.00
16 |   50.00 |     50.00
17 |   66.24 |     53.00
18 |   50.00 |     50.00
19 |   83.15 |     62.33
20 |   50.00 |     50.00
21 |    0.00 |      0.00
22 |   56.21 |     50.35
23 |    0.00 |      0.00
24 |   50.00 |     50.00
25 |    0.00 |      0.00
26 |   50.00 |     50.00
27 |    0.00 |      0.00
28 |   50.00 |     50.00
29 |   17.21 |     44.67
30 |   82.26 |     58.33
31 |   16.08 |     42.00
32 |   81.87 |     56.67
33 |   15.55 |     40.67
34 |   15.05 |     39.33
35 |   14.17 |     36.67
36 |   83.51 |     64.00
37 |   83.01 |     61.67
38 |   81.95 |     

C:\Users\evely\AppData\Local\Temp\ipykernel_20560\3863122705.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  id_p = int(row[0])
C:\Users\evely\AppData\Local\Temp\ipykernel_20560\3863122705.py:158: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gaji = float(row[1])
C:\Users\evely\AppData\Local\Temp\ipykernel_20560\3863122705.py:159: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cicilan = float(row[2])


In [3]:
# FUNGSI KEANGGOTAAN
def trapezoidal(x, a, b, c, d):
    if x <= a or x >= d:
        return 0.0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x <= c:
        return 1.0
    elif c < x < d:
        return (d - x) / (d - c)
    else:
        return 0.0

def triangular(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x < c:
        return (c - x) / (c - b)
    else:
        return 0.0


# FUZZIFIKASI INPUT
def fuzzify_ipk(ipk):
    return {
        "buruk": trapezoidal(ipk, 0, 0, 2.0, 2.75),
        "cukup": triangular(ipk, 2.0, 2.75, 3.25),
        "bagus": trapezoidal(ipk, 2.75, 3.25, 4.0, 4.0),
    }

def fuzzify_toefl(toefl):
    return {
        "rendah": trapezoidal(toefl, 0, 0, 200, 300),
        "menengah": triangular(toefl, 250, 300, 350),
        "tinggi": trapezoidal(toefl, 300, 400, 700, 700),
    }

def fuzzify_penghasilan(p):
    return {
        "kecil": trapezoidal(p, 0, 0, 3_000_000, 4_000_000),
        "sedang": trapezoidal(p, 3_000_000, 4_000_000, 5_000_000, 6_000_000),
        "besar": trapezoidal(p, 5_000_000, 6_000_000, 7_000_000, 12_000_000),
        "sangat_besar": trapezoidal(
            p, 7_000_000, 12_000_000, float('inf'), float('inf')
        ),
    }


# KEANGGOTAAN OUTPUT
def kelayakan_rendah(x):
    if x <= 40:
        return 1
    elif 40 < x < 70:
        return (70 - x) / 30
    else:
        return 0

def kelayakan_tinggi(x):
    if x <= 40:
        return 0
    elif 40 < x < 70:
        return (x - 40) / 30
    else:
        return 1


# INFERENSI MAMDANI
def inferensi_mamdani(ipk_fz, toefl_fz, pgh_fz):
    domain = range(0, 101)  # 0-100 untuk nilai kelayakan
    agregasi = [0.0] * 101

    # Gabungan penghasilan untuk rule
    kecil_sedang = max(pgh_fz["kecil"], pgh_fz["sedang"])
    besar_semua = max(pgh_fz["besar"], pgh_fz["sangat_besar"])

    # R1: IPK BAGUS & (PENGHASILAN KECIL/Sedang) -> TINGGI
    alpha = min(ipk_fz["bagus"], kecil_sedang)
    for x in domain:
        agregasi[x] = max(agregasi[x], min(alpha, kelayakan_tinggi(x)))

    # R2: IPK BAGUS & (PENGHASILAN BESAR/SANGAT BESAR) -> RENDAH
    alpha = min(ipk_fz["bagus"], besar_semua)
    for x in domain:
        agregasi[x] = max(agregasi[x], min(alpha, kelayakan_rendah(x)))

    # R3: IPK CUKUP & (PENGHASILAN KECIL/Sedang) -> RENDAH
    alpha = min(ipk_fz["cukup"], kecil_sedang)
    for x in domain:
        agregasi[x] = max(agregasi[x], min(alpha, kelayakan_rendah(x)))

    # R4: TOEFL TINGGI & (PENGHASILAN KECIL/Sedang) -> TINGGI
    alpha = min(toefl_fz["tinggi"], kecil_sedang)
    for x in domain:
        agregasi[x] = max(agregasi[x], min(alpha, kelayakan_tinggi(x)))

    # R5: TOEFL MENENGAH & (PENGHASILAN KECIL/Sedang) -> TINGGI
    alpha = min(toefl_fz["menengah"], kecil_sedang)
    for x in domain:
        agregasi[x] = max(agregasi[x], min(alpha, kelayakan_tinggi(x)))

    # Defuzzifikasi Centroid
    pembilang = sum(x * agregasi[x] for x in domain)
    penyebut = sum(agregasi)

    return pembilang / penyebut if penyebut != 0 else 0.0


# TOP 10 TERENDAH
def top_10_terendah(hasil, nama_metode="Mamdani"):
    hasil_sorted = sorted(hasil, key=lambda x: x[1])
    print(f"\n10 Nilai Kelayakan Terendah - {nama_metode}")
    print("-" * 40)
    print("NIM | Nilai")
    print("-" * 40)
    for i in range(min(10, len(hasil_sorted))):
        print(f"{hasil_sorted[i][0]:3} | {hasil_sorted[i][1]:6.2f}")


# MAIN PROGRAM
def main():
    data = [
        (1, 450, 4, 750_000),
        (2, 480, 3, 1_500_000),
        (3, 360, 3, 1_255_000),
        (4, 270, 2, 1_040_000),
        (5, 420, 4, 950_000),
        (6, 390, 4, 1_600_000),
        (7, 370, 3, 1_250_000),
        (8, 255, 3, 550_000),
        (9, 325, 2, 735_000),
        (10, 250, 1, 860_000),
    ]

    hasil_mamdani = []

    print("NIM | Nilai Kelayakan (Mamdani)")
    print("-" * 35)
    for nim, toefl, ipk, penghasilan in data:
        ipk_fz = fuzzify_ipk(ipk)
        toefl_fz = fuzzify_toefl(toefl)
        pgh_fz = fuzzify_penghasilan(penghasilan)

        nilai = inferensi_mamdani(ipk_fz, toefl_fz, pgh_fz)
        hasil_mamdani.append((nim, nilai))
        print(f"{nim:3} | {nilai:8.2f}")

    top_10_terendah(hasil_mamdani)

if __name__ == "__main__":
    main()


NIM | Nilai Kelayakan (Mamdani)
-----------------------------------
  1 |    76.93
  2 |    58.54
  3 |    52.28
  4 |    73.14
  5 |    76.93
  6 |    76.35
  7 |    54.21
  8 |    50.00
  9 |    73.82
 10 |     0.00

10 Nilai Kelayakan Terendah - Mamdani
----------------------------------------
NIM | Nilai
----------------------------------------
 10 |   0.00
  8 |  50.00
  3 |  52.28
  7 |  54.21
  2 |  58.54
  4 |  73.14
  9 |  73.82
  6 |  76.35
  1 |  76.93
  5 |  76.93
